In [2]:
import pandas as pd   #Import Pandas Library 
import numpy as np      #Import Numpy library

In [136]:
jsonpath = 'purchase_data.json'   #Assign file path to a variable
new_df = pd.read_json(jsonpath)   #Import file into a dataframe
new_df.head()                     #Print first 5 rows of dataframe for validation

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [137]:
df_purchases = new_df["Item Name"].count()             #Count total rows to see all purchases
df_players = new_df["SN"].nunique()                    #Count unique number of players
df_items = new_df["Item ID"].nunique()                 #Count unique number of items
df_average_price = round(new_df["Price"].mean(),2)     #Find average price
df_total_revenue = new_df["Price"].sum()               #Find total revenue

In [138]:
#Create dataframe with total number of players
players_df = pd.DataFrame({"Player Count": [df_players]})
players_df

,Player Count
0,573


In [139]:
# Create a summary Dataframe using above values
pur_analysis_df = pd.DataFrame({"Number of unique items": [df_items],
                           "Average Price": "$" + str(df_average_price),
                           "Number of purchases": [df_purchases],
                           "Total Revenue": "$" + str(df_total_revenue)})
pur_analysis_df

,Average Price,Number of purchases,Number of unique items,Total Revenue
0,$2.93,780,183,$2286.33


In [140]:
#Create a new datafram with unique players. Keep last value
unique_player_df = new_df.drop_duplicates(["SN"], keep = 'last')

#Count the number of male/female/undisclosed with value counts and use reset index to create dataframe
gender_cnt_df = unique_player_df["Gender"].value_counts().reset_index()

#Add new Column with percentage values
gender_cnt_df['Percentage of players'] = round(gender_cnt_df['Gender']/df_players*100,2)

#Rename the columns
gender_cnt_df.rename(columns = {'index': 'Gender', 'Gender': 'Total Count'}, inplace = True)

#Set Gender as index to display data as per homework instructions
gender_cnt_df.set_index(["Gender"], inplace = True)

#Display PErcentage with correct format
gender_cnt_df["Percentage of players"] = gender_cnt_df["Percentage of players"].map("{:.2f}%".format)

In [141]:
gender_cnt_df

,Total Count,Percentage of players
Gender,,
Male,465,81.15%
Female,100,17.45%
Other / Non-Disclosed,8,1.40%


In [144]:
# Purchase analysis (Gender)
purch_anlys_avg_by_gender = new_df.groupby("Gender").Price.mean().reset_index()

#Calculate Total purchase value by gender
purch_anlys_tot_by_gender = new_df.groupby("Gender").Price.sum().reset_index()

#Merge Purchase analysis with total purchase dataframe on gender
merge1 = pd.merge(purch_anlys_avg_by_gender, purch_anlys_tot_by_gender, on="Gender")

#Set Gender as index value
merge1.set_index(["Gender"], inplace = True)

#Merge total counts dataframe with total price and average price dataframe
merge2 = merge1.merge(gender_cnt_df, how = "inner", left_index = True, right_index = True)

#Calculate normalized Totals
merge2['Normalized Totals'] = merge2['Price_y']/merge2['Total Count']

#Format and rename columns to crate final gender purchase analysis dataframe
merge2.rename(columns = {'Price_x': 'Average Purchase Price', 'Price_y': 'Total Purchase Value'}, inplace = True)
merge2.drop(columns = ['Percentage of players'], inplace = True)
merge2["Average Purchase Price"] = merge2["Average Purchase Price"].map("${:.2f}".format)
merge2["Total Purchase Value"] = merge2["Total Purchase Value"].map("${:.2f}".format)
merge2["Normalized Totals"] = merge2["Normalized Totals"].map("${:.2f}".format)
merge2.head()



,Average Purchase Price,Total Purchase Value,Total Count,Normalized Totals
Gender,,,,
Female,$2.82,$382.91,100,$3.83
Male,$2.95,$1867.68,465,$4.02
Other / Non-Disclosed,$3.25,$35.74,8,$4.47


In [19]:
unique_player_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59
5,20,Male,10,Sleepwalker,1.73,Tanimnya91
6,20,Male,153,Mercenary Sabre,4.57,Undjaskla97


In [147]:
# Purchase Analysis Age
#Find % of players
unique_player_df.count()

#Create bins
bins = [0, 9, 14, 19, 24, 29, 34, 39, 49]

#Create labels
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#Assign bin values to ranges
pd.cut(unique_player_df["Age"], bins, labels = group_labels)

#Assign bins and ranges to unique player dataframe
unique_player_df.loc[:, "Age Range"] = pd.cut(unique_player_df["Age"], bins, labels = group_labels)

#Create dataframe with age range and count unique usernames
age_analysis_df = unique_player_df.groupby("Age Range").SN.count().reset_index()

#Add new Column with percentage values
age_analysis_df['Percentage of players'] = round(age_analysis_df['SN']/df_players*100,2)

#Rename the columns
age_analysis_df.rename(columns = {'SN': 'Total Count'}, inplace = True)

#Assign age range as index
age_analysis_df.set_index(["Age Range"], inplace = True)

#Print Age Demographics Final
age_analysis_df.head(10)

C:\Users\robin\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Total Count,Percentage of players
Age Range,,
<10,19,3.32
10-14,23,4.01
15-19,100,17.45
20-24,259,45.20
25-29,87,15.18
30-34,47,8.20
35-39,27,4.71
40+,11,1.92


In [148]:
#Purchase analysis by Age
#Add bin information defined above to original new_df dataframe with all rows
pd.cut(new_df["Age"], bins, labels = group_labels)
new_df.loc[:, "Age Range"] = pd.cut(new_df["Age"], bins, labels = group_labels)

#Calculate count for each age group
purch_by_age_count = new_df.groupby("Age Range").Price.count().reset_index()

#Calculate total purchase value for each age group
purch_by_age_sum = new_df.groupby("Age Range").Price.sum().reset_index()

#Merge the count and purchase value dataframes
purch_by_age_merged = purch_by_age_count.merge(purch_by_age_sum, how = 'inner', left_index = True, right_index= True)

#Calculate Average purchase price
purch_by_age_merged["Average Purchase Price"] = round(purch_by_age_merged['Price_y']/purch_by_age_merged['Price_x'],2)

#Drop extra columns and rename other columns in previous dataframe
purch_by_age_merged.drop(['Age Range_y'], axis = 1, inplace = True)

#Rename columns as per instructions in PDF document in Homework section
purch_by_age_merged.rename(columns = {'Age Range_x': 'Age Range', 'Price_x': 'Purchase Count', 'Price_y': 'Total Purchase Value'}, inplace = True)

# Set Age Range as index
purch_by_age_merged.set_index(["Age Range"], inplace = True)

#Merge Age Demographics dataframe to Purchase analysis (by age) dataframe
purch_by_age_final = purch_by_age_merged.merge(age_analysis_df, how = "inner", left_index = True, right_index = True)

#Calculate Normalized Total
purch_by_age_final['Normalized Totals'] = round(purch_by_age_final['Total Purchase Value']/purch_by_age_final['Total Count'],2)

#Drop extra columns and create final Purchase analysis by age dataframe
purch_by_age_final.drop(['Total Count', 'Percentage of players'], axis=1, inplace = True)

#Format amount column values with $ signs
purch_by_age_final["Total Purchase Value"] = purch_by_age_final["Total Purchase Value"].map("${:.2f}".format)
purch_by_age_final["Average Purchase Price"] = purch_by_age_final["Average Purchase Price"].map("${:.2f}".format)
purch_by_age_final["Normalized Totals"] = purch_by_age_final["Normalized Totals"].map("${:.2f}".format)

#Print final purchase analysis by gender dataframe
purch_by_age_final.head(10)


,Purchase Count,Total Purchase Value,Average Purchase Price,Normalized Totals
Age Range,,,,
<10,28,$83.46,$2.98,$4.39
10-14,35,$96.95,$2.77,$4.22
15-19,133,$386.42,$2.91,$3.86
20-24,336,$978.77,$2.91,$3.78
25-29,125,$370.33,$2.96,$4.26
30-34,64,$197.25,$3.08,$4.20
35-39,42,$119.40,$2.84,$4.42
40+,17,$53.75,$3.16,$4.89


In [149]:
#Create a data frame form original dataframe to count # of purchases and sum of purchases per player
purch_by_player = new_df.groupby("SN").agg({'Item ID': 'count', 'Price': 'sum'}).reset_index().rename(columns={'Item ID': 'Purchase Count', 'Price': 'Total Purchase Value'})

#Calculate avrage price spent per player by dividing total purchase value by purchase count 
purch_by_player["Average Purchase Price"] = round(purch_by_player['Total Purchase Value']/purch_by_player['Purchase Count'],2)

#Sort player purchase dataframe by total purchase price to get top spenders
purch_by_player.sort_values(by='Total Purchase Value', ascending=False, inplace = True)

#Set SN as index
purch_by_player.set_index(["SN"], inplace = True)

#Format amount column values with $ signs
purch_by_player["Total Purchase Value"] = purch_by_player["Total Purchase Value"].map("${:.2f}".format)
purch_by_player["Average Purchase Price"] = purch_by_player["Average Purchase Price"].map("${:.2f}".format)

#Print top 5 spenders
purch_by_player.head(5)

,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Undirrala66,5,$17.06,$3.41
Saedue76,4,$13.56,$3.39
Mindimnya67,4,$12.74,$3.18
Haellysu29,3,$12.73,$4.24
Eoda93,3,$11.58,$3.86


In [166]:
#Create a data frame to count items by amount sold
items_info1 = new_df.groupby("Item ID").agg({'Price': 'count'}).reset_index().rename(columns={'Price': 'Purchase Count'})

#Create data fram to find total purchase value by items
items_info2 = new_df.groupby("Item ID").agg({'Price': 'sum'}).reset_index().rename(columns={'Price': 'Total Purchase Value'})

#Create a new datafram with unique items
items_info3 = new_df.drop_duplicates(["Item ID"], keep = 'last')

#Keep Desired columns for final display output
items_info4 = items_info3[['Item ID', 'Item Name', 'Price']]

#Merge all dataframes to get desired information
merge_data1 = pd.merge(items_info4, items_info1, on="Item ID", how = 'inner')
item_info_final = pd.merge(merge_data1, items_info2, on="Item ID", how = 'inner')

#Sort player purchase dataframe by total purchase count to get top items
item_info_final.sort_values(by='Purchase Count', ascending=False, inplace = True)

#Set Item ID and Item Name as index
item_info_final.set_index(["Item ID", "Item Name"], inplace = True)

#Sort item info dataframe by descending total count and purchase value to get most popular items
item_info_final.sort_values(['Purchase Count', 'Total Purchase Value'] , ascending=[False, False], inplace = True)

#Format amount column values with $ signs
item_info_final["Total Purchase Value"] = item_info_final["Total Purchase Value"].map("${:.2f}".format)
item_info_final["Price"] = item_info_final["Price"].map("${:.2f}".format)

#Print most popular items!
item_info_final.head(10)

,,Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",$2.35,11,$25.85
84,Arcane Gem,$2.23,11,$24.53
34,Retribution Axe,$4.14,9,$37.26
31,Trickster,$2.07,9,$18.63
13,Serenity,$1.49,9,$13.41
175,Woeful Adamantite Claymore,$1.24,9,$11.16
107,"Splitter, Foe Of Subtlety",$3.61,8,$28.88
152,Darkheart,$3.15,8,$25.20
44,Bonecarvin Battle Axe,$2.46,8,$19.68


In [169]:
#Create a data frame to count items by amount sold
item_info1 = new_df.groupby("Item ID").agg({'Price': 'count'}).reset_index().rename(columns={'Price': 'Purchase Count'})

#Create data fram to find total purchase value by items
item_info2 = new_df.groupby("Item ID").agg({'Price': 'sum'}).reset_index().rename(columns={'Price': 'Total Purchase Value'})

#Create a new datafram with unique items
item_info3 = new_df.drop_duplicates(["Item ID"], keep = 'last')

#Keep Desired columns for final display output
item_info4 = item_info3[['Item ID', 'Item Name', 'Price']]

#Merge all dataframes to get desired information
merge_data1 = pd.merge(item_info4, item_info1, on="Item ID", how = 'inner')
itemz_info_final = pd.merge(merge_data1, item_info2, on="Item ID", how = 'inner')

#Sort player purchase dataframe by total purchase count to get top items
itemz_info_final.sort_values(by='Purchase Count', ascending=False, inplace = True)

#Set Item ID and Item Name as index
itemz_info_final.set_index(["Item ID", "Item Name"], inplace = True)

#Sort item info dataframe by purchase value to get most popular items
itemz_info_final.sort_values(['Total Purchase Value'] , ascending=False, inplace = True)

#Format amount column values with $ signs
itemz_info_final["Total Purchase Value"] = itemz_info_final["Total Purchase Value"].map("${:.2f}".format)
itemz_info_final["Price"] = itemz_info_final["Price"].map("${:.2f}".format)

#Print most profitable items!
itemz_info_final.head(10)


,,Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,$4.14,9,$37.26
115,Spectral Diamond Doomblade,$4.25,7,$29.75
32,Orenmir,$4.95,6,$29.70
103,Singed Scalpel,$4.87,6,$29.22
107,"Splitter, Foe Of Subtlety",$3.61,8,$28.88
101,Final Critic,$4.62,6,$27.72
7,"Thorn, Satchel of Dark Souls",$4.51,6,$27.06
145,Fiery Glass Crusader,$4.45,6,$26.70
39,"Betrayal, Whisper of Grieving Widows",$2.35,11,$25.85
